In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

## 1) Dataset

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', cache=True)

X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

#Nomralization of the value between 0 and 1
X /= 255.0

#Dataset for Cnn model
X = X.reshape(-1, 1, 28, 28) #Comment if you want to use the FC model

#Train - test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((52500, 1, 28, 28), (52500,), (17500, 1, 28, 28), (17500,))

## 2) Models

### 2.1) FC3

In [5]:
#FULLY CONNECTED
class BasicNet(nn.Module):
    
    def __init__(self, Ni, Nh1, Nh2, No):
        """
        Ni - Input size
        Nh1 - Neurons in the 1st hidden layer
        Nh2 - Neurons in the 2nd hidden layer
        No - Output size
        """
        super().__init__()
        
        self.fc1 = nn.Linear(in_features=Ni, out_features=Nh1)
        self.fc2 = nn.Linear(in_features=Nh1, out_features=Nh2)
        self.out = nn.Linear(in_features=Nh2, out_features=No)
        self.soft = nn.Softmax()
        
        self.relu= nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.name="BasicNet"

        print('Network initialized')
        
    def forward(self, input, additional_out=False):
        x=input
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.out(x)
        x= self.soft(x)
        return x

### 2.2) Convolutional Network

In [3]:
#CONVOLUTIONAL NN
import torch.nn.functional as F
class Cnn(nn.Module):
    def __init__(self, dropout=0.5,conv1=32,conv2=64,fc1=128):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, conv1, kernel_size=3)
        self.conv2 = nn.Conv2d(conv1, conv2, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout)
        self.fc1 = nn.Linear(1600, fc1) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(fc1, 10)
        self.fc1_drop = nn.Dropout(p=dropout)

    def forward(self, x):
        x = torch.relu(F.max_pool2d(self.conv1(x), 2))
        x = torch.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        # flatten over channel, height and width = 1600
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        
        x = torch.relu(self.fc1_drop(self.fc1(x)))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

## 3) Training

### 3.1) Initialization

In [9]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

# Define the loss function
loss_function = nn.CrossEntropyLoss()

#Early Stopping
from skorch.callbacks import EarlyStopping

my_early = EarlyStopping(
    monitor='valid_loss',
    patience=10,
    threshold=0.0001,
    threshold_mode='rel',
    lower_is_better=True)

#Model initialization
from skorch import NeuralNetClassifier

#FULLY CONNECTED
net = NeuralNetClassifier(
    module=BasicNet,
    module__Ni= 784,
    module__Nh1 = 8,
    module__Nh2 = 48,
    module__No = 10,
    max_epochs=50,
    
    device=device,  # uncomment this to train with CUDA
    #lr=0.1,
    #optimizer = optim.SGD,
    optimizer = optim.Adam,
    optimizer__lr=0.001,
    optimizer__weight_decay=1e-5, #L2 norm
    criterion=nn.CrossEntropyLoss,
    callbacks = [my_early],
    #verbose=0
)

#CNN
net = NeuralNetClassifier(
    module=Cnn,
    module__conv1=32,
    module__conv2=32,
    module__fc1=32,
    max_epochs=50,
    #lr=0.002,
    device=device,
    optimizer = optim.Adam,
    optimizer__lr=0.002,
    optimizer__weight_decay=1e-5, #L2 norm
    criterion=nn.CrossEntropyLoss,
    callbacks = [my_early],
)

Training device: cuda


### 3.2) Grid Search and Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

#FULLY CONNECTED
params = {
    'module__Nh1': [8,16,32,48],
    'module__Nh2': [8,16,32,48],
    'max_epochs': [1500],
    'optimizer__lr':[0.1, 0.01, 0.001],
    'optimizer__weight_decay':[1e-3,1e-4,1e-5] #L2 norm,
}
params = {
    'module__Nh1': [8,48],
    'module__Nh2': [8,48],
    'max_epochs': [1500],
    'optimizer__lr':[0.01, 0.001],
    'optimizer__weight_decay':[1e-4,1e-5] #L2 norm,
}

#CNN
params = {
    'module__Nh1': [8,48],
    'module__Nh2': [8,48],
    'max_epochs': [1500],
    'optimizer__lr':[0.01, 0.001],
    'optimizer__weight_decay':[1e-4,1e-5] #L2 norm,
}


gs = GridSearchCV(net, params, refit=True, cv=3, scoring="neg_mean_squared_error",verbose=10)

gs.fit(X_train, y_train)

print(gs.best_score_, gs.best_params_)
net=gs.best_estimator_

### 3.3) Normal Training

In [10]:
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.7627       0.9332        1.5304  3.7377
      2        1.5418       0.9645        1.4967  3.5663
      3        1.5233       0.9687        1.4928  3.6830
      4        1.5160       0.9717        1.4893  3.5606
      5        1.5092       0.9772        1.4841  3.6598
      6        1.5066       0.9788        1.4822  3.5958
      7        1.5018       0.9781        1.4827  3.4942
      8        1.4997       0.9791        1.4817  3.4795
      9        1.4990       0.9798        1.4811  3.5110
     10        1.4972       0.9816        1.4797  3.5265
     11        1.4943       0.9818        1.4794  3.4556
     12        1.4961       0.9822        1.4791  3.5175
     13        1.4954       0.9815        1.4794  3.4946
     14        1.4936       0.9797        1.4812  3.4721
     15        1.4935       0.9836        1.4773  3.5300
     16        1.4919       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Cnn(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv2_drop): Dropout2d(p=0.5, inplace=False)
    (fc1): Linear(in_features=1600, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
    (fc1_drop): Dropout(p=0.5, inplace=False)
  ),
)

### 3.4) Plot Losses

In [ ]:
import datetime
save_name="Classification_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

# get train losses from all epochs, a list of floats
history = net.history
train_loss_log=history[:, 'train_loss']
val_loss_log=history[:, 'valid_loss']

# Plot losses
plt.figure(figsize=(12,8))
plt.plot(train_loss_log)
plt.plot(val_loss_log)
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Validation loss'], loc='upper right')
plt.savefig("models/"+save_name+"_Losses", dpi=400)
plt.show()

## 4) Test the model

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = net.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
print(test_acc)

## 5) Metrics Summary

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
						  save_path='models/'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')

    plt.figure(figsize=(15, 15))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.3f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), size=11,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=30)
    plt.xlabel('Predicted label', fontsize=30)
    plt.savefig(save_path+"_picConfMatrix.png", dpi=400)
    plt.tight_layout()

In [ ]:
#Accuracy
val_acc=history[:, 'valid_acc'][-1]
val_loss=history[:, 'valid_loss'][-1]
train_loss=history[:, 'train_loss'][-1]

print("Val Acc:\t",round(val_acc,3))
print("Test Acc:\t",round(float(test_acc),3))

# Precision and Recall(sensitivity/true positive rate)
from sklearn.metrics import precision_score, recall_score
prec=precision_score(y_test, y_pred,average='micro')
rec=recall_score(y_test, y_pred,average='micro')

#F1 - high if both recall and precision are high.
from sklearn.metrics import f1_score
f1=f1_score(y_test, y_pred,average='micro')

print("Precision:\t",round(prec,3))
print("Recall:\t\t",round(rec,3))
print("F1:\t\t",round(f1,3))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
categories=[0,1,2,3,4,5,6,7,8,9]
plot_confusion_matrix(cm,categories, normalize=False,save_path="models/"+save_name)


## 6) Save the model

In [ ]:
#Saving the whole model
import pickle
with open("models/"+save_name+".pkl", 'wb') as f:
    pickle.dump(net, f)
    
#Load the model
#with open(file_name, 'rb') as f:
#    new_net = pickle.load(f)

#Save Metrics to File
f = open("models/"+save_name+"_Metrics.txt", "a")
f.write('Train loss:\t'+ str(round(train_loss,3))+ "\n")
f.write('Val loss:\t'+ str(round(val_loss,3))+ "\n")
f.write('Val acc:\t'+ str(round(val_acc,3))+ "\n")
f.write('Test acc:\t'+ str(round(test_acc,3))+ "\n")
f.write("Precision:\t"+str(round(prec,3))+ "\n")
f.write("Recall:\t\t"+str(round(rec,3))+ "\n")
f.write("F1:\t\t"+str(round(f1,3)))

f.close()

In [ ]:
#Most mispredicted labels
n_mistakes=10
import heapq
h=[]
nCategories=10
for i in range(nCategories):
    for j in range(i+1,nCategories):
        heapq.heappush(h,(cm[i,j]+cm[j,i],(i,j)))
for e in heapq.nlargest(n_mistakes,h):
    print(e[0],e[1][0],"-",e[1][1])

f = open("models/"+save_name+"_Metrics.txt", "a")
f.write('\n\nMost '+str(n_mistakes)+ ' mispredicted labels\n')
for e in heapq.nlargest(n_mistakes,h):
    f.write(str(e[0])+"\t"+str(categories[e[1][0]])+"-"+str(categories[e[1][1]])+"\n")
f.close()

## 7) Network Analysis

### 7.1) Weights histogram

#### 7.1.1) 3FC Network

In [ ]:
#Access network parameters
my_best_net = net.module_

#First hidden Layer
h1_w = my_best_net.fc1.weight.data.cpu().numpy()
h1_b = my_best_net.fc1.bias.data.cpu().numpy()

#Second hidden Layer
h2_w = my_best_net.fc2.weight.data.cpu().numpy()
h2_b = my_best_net.fc2.bias.data.cpu().numpy()

# Output layer
out_w = my_best_net.out.weight.data.cpu().numpy()
out_b = my_best_net.out.bias.data.cpu().numpy()

In [ ]:
# Weights histogram
fig, axs = plt.subplots(3, 1, figsize=(12,8))
axs[0].hist(h1_w.flatten(), 50)
axs[0].set_title('First hidden layer weights')
axs[1].hist(h2_w.flatten(), 50)
axs[1].set_title('Second hidden layer weights')
axs[2].hist(out_w.flatten(), 50)
axs[2].set_title('Output layer weights')
[ax.grid() for ax in axs]
plt.tight_layout()
plt.savefig("models/"+save_name+"_Weights-histogram", dpi=400)
plt.show()

#### 7.1.2) Convolution network

In [ ]:
#Access network parameters
my_best_net = net.module_

#First hidden Layer
h1_w = my_best_net.fc1.weight.data.cpu().numpy()
h1_b = my_best_net.fc1.bias.data.cpu().numpy()

#Second hidden Layer
h2_w = my_best_net.fc2.weight.data.cpu().numpy()
h2_b = my_best_net.fc2.bias.data.cpu().numpy()

# Conv 1 layer
c1_w = my_best_net.conv1.weight.data.cpu().numpy()
c1_b = my_best_net.conv1.bias.data.cpu().numpy()

# Conv 2 layer
c2_w = my_best_net.conv2.weight.data.cpu().numpy()
c2_b = my_best_net.conv2.bias.data.cpu().numpy()


In [ ]:
# Weights histogram
fig, axs = plt.subplots(4, 1, figsize=(12,8))
axs[0].hist(c1_w.flatten(), 50)
axs[0].set_title('Conv1 layer weights')
axs[1].hist(c2_w.flatten(), 50)
axs[1].set_title('Conv2 layer weights')
axs[2].hist(h1_w.flatten(), 50)
axs[2].set_title('First FC layer weights')
axs[3].hist(h2_w.flatten(), 50)
axs[3].set_title('Second FC layer weights')

[ax.grid() for ax in axs]
plt.tight_layout()
plt.savefig("models/"+save_name+"_Weights-histogram", dpi=400)
plt.show()

### 7.2) Analyze activations

#### 7.2.1) 3FC

In [ ]:
??????????????????
ù"""def get_activation(layer, input, output):
    global activation
    activation = torch.softmax(output)
    
### Register hook
net=my_best_net
hook_handle = net.fc2.register_forward_hook(get_activation)


### Analyze activations
net = net.to(device)
net.eval()
with torch.no_grad():
    x1 = torch.from_numpy(X_test[0]).to(device)
    y1 = net(x1)
    z1 = activation
    x2 = torch.tensor(X_test[1]).float().to(device)
    y2 = net(x2)
    z2 = activation
    x3 = torch.tensor(X_test[2]).float().to(device)
    y3 = net(x3)
    z3 = activation

### Remove hook
hook_handle.remove()

### Plot activations
fig, axs = plt.subplots(3, 1, figsize=(12,6))
axs[0].stem(z1.cpu().numpy(), use_line_collection=True)
axs[0].set_title('Last layer activations for input x=%.2f' % x1)
axs[1].stem(z2.cpu().numpy(), use_line_collection=True)
axs[1].set_title('Last layer activations for input x=%.2f' % x2)
axs[2].stem(z3.cpu().numpy(), use_line_collection=True)
axs[2].set_title('Last layer activations for input x=%.2f' % x3)
plt.tight_layout()
plt.savefig("models/"+save_name+"_Activations", dpi=400)
plt.show()"""

#### 7.2.2) Convolution Network

In [ ]:
#......

### 7.3) Receptive fields

In [ ]:

"""
linear combination method for visualising the features discussed in class, which is straightforward for the fully-connected model. Then if you wish you can explore more sophisticated methods, such as the method that allows to create an "optimal" image that maximally activates the neuron. This can be "easily" done also for the CNN. In any case, receptive fields are meaningful only for the classification task.

"""

#### 7.3.1) 3FC

In [ ]:
#FULLY CONNECTED NET
#Receptive fields of the last layer
for ii in range(10):
    vis3 = np.matmul(out_w[ii],np.matmul(h2_w,h1_w))
    print("\nLabel:", ii)
    plt.imshow(vis3.reshape(784).reshape(28,28))
    plt.savefig("models/"+save_name+"_Receptive fields_"+str(ii), dpi=400)
    plt.show()

#### 7.3.2) Convolution Network

In [ ]:
#.....